In [1]:
import numpy as np
import pendulum as pend
import pandas as pd

def backhrs(hour):
    if hour <= 40:
        return hour, 0
    else:
        return 40, hour - 40

def genSum(arg):
    pend.week_starts_at(wday=1)

    most_recent = pend.datetime(2022, 7, 18)
    end_date = pend.datetime(2023, 1, 29)

    timfr = pend.period(most_recent, end_date)

    hourly, ot_rate, tax = 24.00, 1.5, .25

    hours_spread = np.random.choice([37.5, 40.0, 42.5, 45.0, 47.5, 50.0], p=[.2, .3, .2, .1, .1, .1],
                                    size=timfr.in_weeks() + 1)
    work_weeks_dict = []

    hrspridx = 0
    for dt in timfr.range('weeks'):
        wk_hours = backhrs(hours_spread[hrspridx])
        reg = wk_hours[0] * hourly
        ot = wk_hours[1] * hourly * ot_rate

        this_week = {
            "Week": 1 + hrspridx,
            "Wk Beg": dt.format('MM-DD-YYYY'),
            "Wk End": dt.add(days=6).format('MM-DD-YYYY'),
            "Reg Hrs": wk_hours[0],
            "OT Hrs": wk_hours[1],
            "Tot Hrs": hours_spread[hrspridx],
            "Reg Pay": reg,
            "OT Pay": ot,
            "Taxes": tax * (ot + reg),
            "Gross": ot + reg,
            "Net": (ot + reg) - ( tax * (ot + reg ) )
        }
        work_weeks_dict.append(this_week)
        hrspridx += 1

    df = pd.DataFrame(work_weeks_dict)

    wk1_idx = 0
    wk2_idx = 1

    pay_periods_dict = []

    while wk1_idx < len(work_weeks_dict):

        wk1 = work_weeks_dict[wk1_idx]
        wk2 = work_weeks_dict[wk2_idx]

        this_pay_period = {
            "Pay Period": pay_periods_dict.__len__() + 1,
            "Wk Beg": wk1.get("Wk Beg"),
            "Wk End": wk2.get("Wk End"),
            "Reg Hrs": wk1.get("Reg Hrs") + wk2.get("Reg Hrs"),
            "OT Hrs": wk1.get("OT Hrs") + wk2.get("OT Hrs"),
            "Tot Hrs": wk1.get("Tot Hrs") + wk2.get("Tot Hrs"),
            "Reg Pay": wk1.get("Reg Pay") + wk2.get("Reg Pay"),
            "OT Pay": wk1.get("OT Pay") + wk2.get("OT Pay"),
            "Taxes": wk1.get("Taxes") + wk2.get("Taxes"),
            "Gross": wk1.get("Gross") + wk2.get("Gross"),
            "Net": wk1.get("Net") + wk2.get("Net"),
        }

        pay_periods_dict.append(this_pay_period)
        wk1_idx += 2
        wk2_idx += 2

    if arg == "Net":
        return pd.DataFrame(pay_periods_dict)['Net'].sum()
    else:
        return pd.DataFrame(pay_periods_dict)


In [2]:
# elem = []
# for a in range(0, 100):
#     sum = genSum(arg="Net")
#     print(sum)
#     elem.append(sum)
#
# pd.DataFrame(elem).mean()

In [3]:
def get_pay_month(active_date):
    date = pend.parse(active_date,strict=False)
    pay_date = date.add(days=3)
    print(pay_date.format('MM-DD-YYYY'))
    print(pay_date.month)
    return pay_date.format("MMMM")

In [4]:
df = genSum(arg="")
df['Month'] = df.apply(lambda row : get_pay_month(row['Wk End']), axis=1)
df

08-03-2022
8
08-17-2022
8
08-31-2022
8
09-14-2022
9
09-28-2022
9
10-12-2022
10
10-26-2022
10
11-09-2022
11
11-23-2022
11
12-07-2022
12
12-21-2022
12
01-04-2023
1
01-18-2023
1
02-01-2023
2


,Pay Period,Wk Beg,Wk End,Reg Hrs,OT Hrs,Tot Hrs,Reg Pay,OT Pay,Taxes,Gross,Net,Month
0,1,07-18-2022,07-31-2022,80.0,5.0,85.0,1920.0,180.0,525.0,2100.0,1575.0,August
1,2,08-01-2022,08-14-2022,80.0,12.5,92.5,1920.0,450.0,592.5,2370.0,1777.5,August
2,3,08-15-2022,08-28-2022,80.0,2.5,82.5,1920.0,90.0,502.5,2010.0,1507.5,August
3,4,08-29-2022,09-11-2022,80.0,10.0,90.0,1920.0,360.0,570.0,2280.0,1710.0,September
4,5,09-12-2022,09-25-2022,77.5,0.0,77.5,1860.0,0.0,465.0,1860.0,1395.0,September
5,6,09-26-2022,10-09-2022,80.0,2.5,82.5,1920.0,90.0,502.5,2010.0,1507.5,October
6,7,10-10-2022,10-23-2022,80.0,17.5,97.5,1920.0,630.0,637.5,2550.0,1912.5,October
7,8,10-24-2022,11-06-2022,77.5,0.0,77.5,1860.0,0.0,465.0,1860.0,1395.0,November
8,9,11-07-2022,11-20-2022,80.0,12.5,92.5,1920.0,450.0,592.5,2370.0,1777.5,November
9,10,11-21-2022,12-04-2022,77.5,0.0,77.5,1860.0,0.0,465.0,1860.0,1395.0,December


In [7]:
df.groupby(['Month'])['Net'].sum()

Month
August       4860.0
December     2857.5
February     1395.0
January      3352.5
November     3172.5
October      3420.0
September    3105.0
Name: Net, dtype: float64